In [1]:
import pandas as pd

import sys
sys.path.append("../src/")

from src import CSV_DELIMITER
from src.fingerprint import Fingerprint
import numpy as np

In [2]:
ATTRIBUTES_TO_EXCLUDE = [Fingerprint.COUNTER, Fingerprint.CREATION_TIME, Fingerprint.END_TIME]

In [3]:
df = pd.read_csv("./data/processed/fingerprint_dataset.csv", delimiter=CSV_DELIMITER, engine="python")

In [48]:
attributes_to_analyze = [attribute for attribute in df.columns.tolist() if attribute not in ATTRIBUTES_TO_EXCLUDE]

for attribute in attributes_to_analyze:
    nb_distinct_values = df[attribute].value_counts().count()
    print("{} : {:d} distinct values".format(attribute, nb_distinct_values))

id : 5808 distinct values
addressHttp : 23303 distinct values
userAgentHttp : 3550 distinct values
acceptHttp : 16 distinct values
connectionHttp : 2 distinct values
encodingHttp : 17 distinct values
languageHttp : 660 distinct values
orderHttp : 201 distinct values
pluginsJSHashed : 5843 distinct values
platformJS : 32 distinct values
cookiesJS : 2 distinct values
dntJS : 2 distinct values
timezoneJS : 35 distinct values
resolutionJS : 595 distinct values
localJS : 2 distinct values
sessionJS : 2 distinct values
canvasJSHashed : 4002 distinct values


fontsFlashHashed : 5361 distinct values
resolutionFlash : 311 distinct values
languageFlash : 27 distinct values
platformFlash : 290 distinct values


In [53]:
for attribute in attributes_to_analyze:
    res = df[Fingerprint].value_counts()
    my_df = pd.DataFrame(res)
    my_df.to_csv(attribute + '_modality.csv', index=False, header=False)
print(my_df)

TypeError: 'type' object is not subscriptable

In [32]:
df[Fingerprint.PLATFORM_JS].dtypes

dtype('O')

In [64]:
for attribute in attributes_to_analyze:
    res = df[attribute].value_counts()
    my_df = pd.DataFrame(res)
    my_df.to_csv("./data/test/"+attribute+"category.csv", index=True)